In [1]:
import heapq
import random
import time
import sys

# Goal-Checking Function
def is_goal(node, goal):
    """
    Checks if the current node is the goal node.
    """
    return node == goal

# Initial Setup
def get_user_input(graph):
    print("Available locations: ", list(graph.keys()))
    start = input("Enter the starting location: ").strip()
    end = "Exit"  # Set goal as "Exit"
    return start, end

# Transition & Cost Matrix
campus_map = {
    "Admission Office": {"Hostel Office": 2, "Library": 4},
    "Hostel Office": {"Admission Office": 2, "Hostel Visit": 2, "Canteen": 6, "Library": 4},
    "Hostel Visit": {"Hostel Office": 2, "Canteen": 6, "Exit": 4},
    "Canteen": {"Hostel Visit": 6, "Hostel Office": 6, "Library": 7, "Dep't Visit": 2, "Exit": 8},
    "Dep't Visit": {"Canteen": 2, "Library": 3, "Exit": 5},
    "Library": {"Admission Office": 4, "Canteen": 7, "Dep't Visit": 3, "Hostel Office": 4},
    "Exit": {"Dep't Visit": 5, "Canteen": 8, "Hostel Visit": 4}
}

# Transition Model/Successor Function
def get_successors(graph, node):
    """
    Returns the successors of a given node as a dictionary of neighbors and costs.
    """
    return graph.get(node, {})

# Heuristic Function
def heuristic(node, goal):
    """
    A heuristic function to estimate the cost from the current node to the goal.
    Currently, it returns 0 for uniform cost search.
    """
    return 0

# A* Algorithm Implementation
def a_star_search(graph, start, goal):
    open_set = []
    heapq.heappush(open_set, (0, start))
    came_from = {}
    g_score = {node: float('inf') for node in graph}
    g_score[start] = 0
    f_score = {node: float('inf') for node in graph}
    f_score[start] = heuristic(start, goal)
    closed_set = set()

    while open_set:
        _, current = heapq.heappop(open_set)

        if is_goal(current, goal):  # Use goal-checking function
            path = []
            total_cost = 0
            while current in came_from:
                path.append(current)
                next_node = came_from[current]
                total_cost += graph[next_node].get(current, 0)
                current = next_node
            path.append(start)
            return path[::-1], total_cost

        closed_set.add(current)

        for neighbor, cost in get_successors(graph, current).items():
            if neighbor in closed_set:
                continue

            tentative_g_score = g_score[current] + cost
            if tentative_g_score < g_score[neighbor]:
                came_from[neighbor] = current
                g_score[neighbor] = tentative_g_score
                f_score[neighbor] = tentative_g_score + heuristic(neighbor, goal)
                heapq.heappush(open_set, (f_score[neighbor], neighbor))

    return None, 0

# Random Restart Hill Climbing Algorithm
def hill_climbing_with_restart(graph, start, goal, max_restarts=10, max_steps=50, stagnation_limit=10):
    best_path = None
    best_cost = float('inf')
    random.seed(42)  # Ensures reproducibility

    for _ in range(max_restarts):
        current = start
        path = [current]
        total_cost = 0
        steps = 0
        stagnation_counter = 0
        visited = set()

        while not is_goal(current, goal) and steps < max_steps:  # Use goal-checking function
            neighbors = list(get_successors(graph, current).items())
            if not neighbors:
                break

            random.shuffle(neighbors)
            next_node, cost = neighbors[0]

            if next_node in visited:
                stagnation_counter += 1
                if stagnation_counter >= stagnation_limit:
                    break
                continue

            path.append(next_node)
            total_cost += cost
            current = next_node
            steps += 1
            stagnation_counter = 0
            visited.add(current)

        if is_goal(current, goal) and total_cost < best_cost:  # Use goal-checking function
            best_cost = total_cost
            best_path = path

    return best_path, best_cost

# Time and Space Complexity Evaluation
def evaluate_algorithm(algorithm, graph, start, goal):
    start_time = time.time()
    path, total_cost = algorithm(graph, start, goal)
    end_time = time.time()
    time_taken = end_time - start_time
    memory_usage = sys.getsizeof(graph) + sys.getsizeof(path)
    return path, total_cost, time_taken, memory_usage

# Main Execution
if __name__ == "__main__":
    start_location, goal_location = get_user_input(campus_map)

    print("\nRunning A* Algorithm...")
    optimal_path, optimal_cost = a_star_search(campus_map, start_location, goal_location)
    if optimal_path:
        print(f"Optimal Path (A*): {optimal_path}, Total Cost: {optimal_cost}")
    else:
        print("No path found for A*.")

    print("\nRunning Random Restart Hill Climbing...")
    best_path, best_cost = hill_climbing_with_restart(campus_map, start_location, goal_location)
    if best_path:
        print(f"Best Path (Hill Climbing): {best_path}, Cost: {best_cost}")
    else:
        print("No valid path found for Hill Climbing.")

    print("\nEvaluating A* Algorithm...")
    _, a_star_cost, a_star_time, a_star_memory = evaluate_algorithm(a_star_search, campus_map, start_location, goal_location)
    print(f"A* - Time: {a_star_time:.4f}s, Memory: {a_star_memory / 1024:.2f} KB, Cost: {a_star_cost}")

    print("\nEvaluating Hill Climbing Algorithm...")
    _, hill_climbing_cost, hill_climbing_time, hill_climbing_memory = evaluate_algorithm(hill_climbing_with_restart, campus_map, start_location, goal_location)
    print(f"Hill Climbing - Time: {hill_climbing_time:.4f}s, Memory: {hill_climbing_memory / 1024:.2f} KB, Cost: {hill_climbing_cost}")


Available locations:  ['Admission Office', 'Hostel Office', 'Hostel Visit', 'Canteen', "Dep't Visit", 'Library', 'Exit']


Enter the starting location:  Admission Office



Running A* Algorithm...
Optimal Path (A*): ['Admission Office', 'Hostel Office', 'Hostel Visit', 'Exit'], Total Cost: 8

Running Random Restart Hill Climbing...
Best Path (Hill Climbing): ['Admission Office', 'Hostel Office', 'Canteen', 'Exit'], Cost: 16

Evaluating A* Algorithm...
A* - Time: 0.0000s, Memory: 0.35 KB, Cost: 8

Evaluating Hill Climbing Algorithm...
Hill Climbing - Time: 0.0000s, Memory: 0.38 KB, Cost: 16
